In [8]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['한국어 교정']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
pattern = f"^쓰는데|쓰는데$|/$|^\"|\"$"
def postprocess(x):
    x = re.sub(pattern, '', x)
    x = x.strip()
    return x
#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            text = line['ko']
            corrected_text = [postprocess(line['corrected'])]
            if 'autoproof' in line:
                corrected_text.append(postprocess(line['autoproof']))
                
            if any([text == corr_text for corr_text in corrected_text]) or len(text.split())<4:
                continue
            #### data preprocess end 
            
            #### 한국어 교정
            ### 'output' is list of output sorted by relevance (1st is most relevant)
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data['input'] = text
            data['output'] = corrected_text
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 한국어 교정 end
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 12/12 [00:00<00:00, 34.49it/s]
